Imports & Logging

In [ ]:
import json
import logging

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


Khởi tạo SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName("BTC Price Z-Score") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5") \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .config("spark.hadoop.fs.hdfs.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .config("spark.driver.extraJavaOptions", "-Dhadoop.home.dir=D:/minhanh/env") \
    .getOrCreate()

# Chỉ show WARN+ để bớt log
spark.sparkContext.setLogLevel("WARN")


Định nghĩa schema và đọc stream giá từ Kafka

# Schema cho dữ liệu giá
price_schema = StructType([
    StructField("symbol",    StringType(), True),
    StructField("price",     DoubleType(), True),
    StructField("timestamp", StringType(), True)
])

price_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "btc-price") \
    .option("startingOffsets", "latest") \
    .load() \
    .select(from_json(col("value").cast("string"), price_schema).alias("data")) \
    .select("data.*") \
    .withColumn("timestamp", to_timestamp("timestamp"))


Định nghĩa schema và đọc stream stats từ Kafka

In [ ]:
# Schema cho dữ liệu thống kê động
stats_schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("symbol",    StringType(), True),
    StructField("stats",     ArrayType(
        StructType([
            StructField("window",     StringType(), True),
            StructField("avg_price",  DoubleType(), True),
            StructField("std_price",  DoubleType(), True)
        ])
    ), True)
])

stats_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "btc-price-moving") \
    .option("startingOffsets", "latest") \
    .load() \
    .select(from_json(col("value").cast("string"), stats_schema).alias("data")) \
    .select("data.*") \
    .withColumn("timestamp", to_timestamp("timestamp"))


Watermark & Join hai stream

In [ ]:
# Watermark để xử lý dữ liệu trễ tối đa 10s
price_df = price_df.withWatermark("timestamp", "10 seconds")
stats_df = stats_df.withWatermark("timestamp", "10 seconds")

# Join trên timestamp và symbol
joined_df = price_df.join(
    stats_df,
    (price_df.timestamp == stats_df.timestamp) &
    (price_df.symbol   == stats_df.symbol),
    "inner"
)


Tính Z-Score & ghi kết quả về Kafka

In [ ]:
# UDF tính điểm Z cho mỗi cửa sổ
def calculate_zscores(price, stats_array):
    result = []
    for stat in stats_array:
        avg_price = stat["avg_price"]
        std_price = stat["std_price"]
        z = (price - avg_price) / std_price if std_price > 0 else 0.0
        result.append({"window": stat["window"], "zscore_price": z})
    return result

calculate_zscores_udf = udf(
    calculate_zscores,
    ArrayType(StructType([
        StructField("window",       StringType(), True),
        StructField("zscore_price", DoubleType(), True)
    ]))
)

# Áp dụng và ghi stream
output_df = joined_df.select(
    price_df.timestamp,
    price_df.symbol,
    calculate_zscores_udf(price_df.price, stats_df.stats).alias("zscores")
)

query = output_df \
    .selectExpr("to_json(struct(*)) AS value") \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "btc-price-zscore") \
    .option("checkpointLocation", "memory") \
    .outputMode("append") \
    .start()

logger.info("Z-Score Streaming started")
query.awaitTermination()
